In [1]:
import numpy as np 
import bagpipes as pipes

from astropy.io import fits

#load the filter curves
def load_goodss(ID):
    """ Load CANDELS GOODS South photometry from the Guo et al. (2013) catalogue. """

    # load up the relevant columns from the catalogue.
    cat = np.loadtxt("hlsp_candels_hst_wfc3_goodss-tot-multiband_f160w_v1-1photom_cat.txt",
                     usecols=(10, 13, 16, 19, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55,
                              11, 14, 17, 20, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56))
    
    # Find the correct row for the object we want.
    row = int(ID) - 1

    # Extract the object we want from the catalogue.
    fluxes = cat[row, :15]
    fluxerrs = cat[row, 15:]

    # Turn these into a 2D array.
    photometry = np.c_[fluxes, fluxerrs]

    # blow up the errors associated with any missing fluxes.
    for i in range(len(photometry)):
        if (photometry[i, 0] == 0.) or (photometry[i, 1] <= 0):
            photometry[i,:] = [0., 9.9*10**99.]
            
    # Enforce a maximum SNR of 20, or 10 in the IRAC channels.
    for i in range(len(photometry)):
        if i < 10:
            max_snr = 20.
            
        else:
            max_snr = 10.
        
        if photometry[i, 0]/photometry[i, 1] > max_snr:
            photometry[i, 1] = photometry[i, 0]/max_snr

    return photometry

goodss_filt_list = np.loadtxt("filters1/goodss_filt_list.txt", dtype="str")

#dictionary for fit instructions
exp = {}                                  
exp["age"] = (0.1, 15.)
exp["tau"] = (0.3, 10.)
exp["massformed"] = (1., 15.)
exp["metallicity"] = (0., 2.5)

dust = {}
dust["type"] = "Calzetti"
dust["Av"] = (0., 2.)

fit_instructions = {}
fit_instructions["redshift"] = (0., 10.)
fit_instructions["exponential"] = exp   
fit_instructions["dust"] = dust

phot = load_goodss(1)  # or 2 or 3
print(type(phot))
print(phot.shape)
print(phot)



In [2]:
IDs = np.arange(1, 4, dtype=int)

fit_cat = pipes.fit_catalogue(IDs, fit_instructions, load_goodss, spectrum_exists=False,
                              cat_filt_list=goodss_filt_list, run="guo_cat")

fit_cat.fit(verbose=False)

In [3]:
#trying this but for my mini cat of n galaxies

import numpy as np 
import bagpipes as pipes
from astropy.table import Table
from astropy.io import fits

#load the filter curves
def load_goodss(ID):
    """ Load CANDELS GOODS South photometry from the Guo et al. (2013) catalogue. """

    # load up the relevant columns from the catalogue.
    cat = Table.read("mini_catalogue_20galaxies.fits") 

    
    # Find the correct row for the object we want.
    row = int(ID) - 1

    filters = Table.read('wavelengths_filters.csv', format='csv')
    # Extract filter names
    filters_list = filters['flux_filter'].tolist()

    # Build column names for fluxes and flux errors
    flux_columns = [f'FLUX_APER_{f}_aper_corr_Jy' for f in filters_list]
    fluxerr_columns = [f'FLUXERR_APER_{f}_loc_depth_10pc_Jy' for f in filters_list]
     # Extract fluxes and errors for this galaxy from the catalogue
    fluxes = np.array([cat[row][col] for col in flux_columns])
    fluxes = np.where(fluxes > 0, fluxes, np.nan)
    fluxerrs = np.array([cat[row][col] if not np.ma.is_masked(cat[row][col]) else np.nan for col in fluxerr_columns])


    # Turn these into a 2D array.
    photometry = np.c_[fluxes, fluxerrs]

    # blow up the errors associated with any missing fluxes.
    for i in range(len(photometry)):
        if (photometry[i, 0] == 0.) or (photometry[i, 1] <= 0) or np.isnan(photometry[i, 0]) or np.isnan(photometry[i, 1]):
            photometry[i,:] = [0., 9.9*10**99.]
            
    # Enforce a maximum SNR of 20, or 10 in the IRAC channels.
    for i in range(len(photometry)):
        if i < 10:
            max_snr = 20.
            
        else:
            max_snr = 10.
        
        if photometry[i, 0]/photometry[i, 1] > max_snr:
            photometry[i, 1] = photometry[i, 0]/max_snr

    return photometry

goodss_filt_list = np.loadtxt("filters1/goodss_filt_list1.txt", dtype="str")

#dictionary for fit instructions
exp = {}                                  
exp["age"] = (0.1, 15.)
exp["tau"] = (0.3, 10.)
exp["massformed"] = (1., 15.)
exp["metallicity"] = (0., 2.5)

dust = {}
dust["type"] = "Calzetti"
dust["Av"] = (0., 2.)

fit_instructions = {}
fit_instructions["redshift"] = (0., 10.)
fit_instructions["exponential"] = exp   
fit_instructions["dust"] = dust


In [4]:
IDs = np.arange(1, 21, dtype=int)

fit_cat = pipes.fit_catalogue(IDs, fit_instructions, load_goodss, spectrum_exists=False,
                              cat_filt_list=goodss_filt_list, run="mini_cat_20")

fit_cat.fit(verbose=False)


Bagpipes: fitting object 1

 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    6
 *****************************************************

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the pr

Traceback (most recent call last):
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/fit.py", line 396, in fit
    self.posterior.get_advanced_quantities()
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/posterior.py", line 204, in get_advanced_quantities
    self.model_galaxy = model_galaxy(self.fitted_model.model_components,
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 207, in __init__
    self.update(model_components, extra_model_components = extra_model_components)
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 410, in update
    self._calculate_xi_ion_caseB(model_components, frame = frame)
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 829, in _calculate_xi_ion_caseB
    self._calculate_ndot_ion_caseB(model_comp, frame = frame, out_units = u.Hz)


Bagpipes: 1 out of 20 objects completed.

Bagpipes: fitting object 2

 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    6
 *****************************************************

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1 

Traceback (most recent call last):
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/fit.py", line 396, in fit
    self.posterior.get_advanced_quantities()
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/posterior.py", line 204, in get_advanced_quantities
    self.model_galaxy = model_galaxy(self.fitted_model.model_components,
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 207, in __init__
    self.update(model_components, extra_model_components = extra_model_components)
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 410, in update
    self._calculate_xi_ion_caseB(model_components, frame = frame)
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 829, in _calculate_xi_ion_caseB
    self._calculate_ndot_ion_caseB(model_comp, frame = frame, out_units = u.Hz)



Bagpipes: fitting object 3

 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    6
 *****************************************************

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the pr

...

Traceback (most recent call last):
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/fit.py", line 396, in fit
    self.posterior.get_advanced_quantities()
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/posterior.py", line 204, in get_advanced_quantities
    self.model_galaxy = model_galaxy(self.fitted_model.model_components,
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 207, in __init__
    self.update(model_components, extra_model_components = extra_model_components)
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 410, in update
    self._calculate_xi_ion_caseB(model_components, frame = frame)
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 829, in _calculate_xi_ion_caseB
    self._calculate_ndot_ion_caseB(model_comp, frame = frame, out_units = u.Hz)


Bagpipes: 18 out of 20 objects completed.

Bagpipes: fitting object 19

 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    6
 *****************************************************

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            

Traceback (most recent call last):
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/fit.py", line 396, in fit
    self.posterior.get_advanced_quantities()
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/posterior.py", line 204, in get_advanced_quantities
    self.model_galaxy = model_galaxy(self.fitted_model.model_components,
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 207, in __init__
    self.update(model_components, extra_model_components = extra_model_components)
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 410, in update
    self._calculate_xi_ion_caseB(model_components, frame = frame)
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 829, in _calculate_xi_ion_caseB
    self._calculate_ndot_ion_caseB(model_comp, frame = frame, out_units = u.Hz)



Bagpipes: fitting object 20

 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    6
 *****************************************************
Rank 0 for 20 ln(ev)=   536.72552532276904      +/-  0.16830661445119424     
 Total Likelihood Evaluations:        48117
 Sampling finished. Exiting MultiNest
 True

Completed in 85.6 seconds.

Generating 500 samples from posterior. Total samples: 1952
'nebular'
Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
dust:Av                        0.130      0.334      0.604
exponential:age                0.987      2.051      3.218
exponential:massformed         2.983      3.096      3.207
exponential:metallicity        0.109      0.294      0.6

Traceback (most recent call last):
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/fit.py", line 396, in fit
    self.posterior.get_advanced_quantities()
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/posterior.py", line 204, in get_advanced_quantities
    self.model_galaxy = model_galaxy(self.fitted_model.model_components,
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 207, in __init__
    self.update(model_components, extra_model_components = extra_model_components)
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 410, in update
    self._calculate_xi_ion_caseB(model_components, frame = frame)
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py", line 829, in _calculate_xi_ion_caseB
    self._calculate_ndot_ion_caseB(model_comp, frame = frame, out_units = u.Hz)
